In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.precision', 4)

print("=" * 70)
print("SRK/T2 IOL FORMULA IMPLEMENTATION")
print("=" * 70)

print("✓ Libraries loaded successfully")

# Load data
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Data')

print(f"Number of patients: {len(df)}")
print()

# Calculate average K
df['K_avg'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

SRK/T2 IOL FORMULA IMPLEMENTATION
✓ Libraries loaded successfully
Number of patients: 96



In [2]:
def calculate_SRKT2(AL, K_avg, IOL_power, A_constant, nc=1.333, k_index=1.3375):
    """
    SRK/T2 Formula (Sheard et al. 2010)
    Modified version of SRK/T formula
    
    Parameters:
    -----------
    AL : float - Axial length (mm)
    K_avg : float - Average keratometry (D)
    IOL_power : float - IOL power (D)
    A_constant : float - A-constant for the IOL
    nc : float - Corneal refractive index (default 1.333)
    k_index : float - Keratometric index (default 1.3375)
    
    Returns:
    --------
    float - Predicted postoperative refraction (D)
    """
    # Constants
    na = 1.336  # Aqueous/vitreous refractive index
    V = 12      # Vertex distance (mm)
    ncm1 = nc - 1
    
    # Calculate corneal radius from keratometry
    r = (k_index - 1) * 1000 / K_avg
    
    # Axial length correction for long eyes
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    # H2 calculation (corneal height) - Sheard's modification
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    # ACD (Anterior Chamber Depth) estimation
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336
    ACD_est = H2 + offset
    
    # Retinal thickness correction
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK  # Optical axial length
    
    # SRK/T2 refraction calculation
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

print("=" * 70)
print("SRK/T2 FORMULA (Sheard et al. 2010)")
print("=" * 70)
print()
print("📐 MAIN FORMULA:")
print("-" * 70)
print()
print("         1000·nₐ·(nₐ·r - nc₋₁·Lopt) - P·(Lopt - ACDest)·(nₐ·r - nc₋₁·ACDest)")
print("REF = ───────────────────────────────────────────────────────────────────────────")
print("       nₐ·(V·(nₐ·r - nc₋₁·Lopt) + Lopt·r) - 0.001·P·(Lopt - ACDest)·(V·(nₐ·r - nc₋₁·ACDest) + ACDest·r)")
print()
print()
print("📖 VARIABLE DEFINITIONS:")
print("=" * 70)
print()
print("INPUT VARIABLES:")
print("-" * 35)
print("• AL         → Axial length of the eye (mm)")
print("• K_avg      → Average keratometry [(Ks + Kf)/2] (diopters)")
print("• IOL_power  → Implanted intraocular lens power (diopters)")
print("• A_constant → IOL-specific A-constant (dimensionless)")
print()
print("PHYSICAL CONSTANTS:")
print("-" * 35)
print("• nₐ = 1.336     → Refractive index of aqueous and vitreous")
print("• nc = 1.333     → Corneal refractive index")
print("• nc₋₁ = 0.333   → nc - 1 (corneal refractive power)")
print("• k_index = 1.3375 → Keratometric index (for K to radius conversion)")
print("• V = 12 mm      → Vertex distance (spectacle-cornea distance)")
print()
print("CALCULATED VARIABLES:")
print("-" * 35)
print("• r          → Corneal radius of curvature (mm)")
print("• LCOR       → Corrected axial length for long eyes (mm)")
print("• H2         → Corneal height according to Sheard (mm)")
print("• ACD_const  → ACD constant derived from A-constant")
print("• offset     → Offset for ACD calculation")
print("• ACDest     → Estimated postoperative anterior chamber depth (mm)")
print("• RETHICK    → Calculated retinal thickness (mm)")
print("• Lopt       → Optical axial length [AL + RETHICK] (mm)")
print("• REF        → Predicted postoperative refraction (diopters)")
print()
print("OTHER SYMBOLS:")
print("-" * 35)
print("• P          → IOL_power (IOL power)")
print("• Ks         → Keratometry flattest meridian (diopters)")
print("• Kf         → Keratometry steepest meridian (diopters)")
print()
print()
print("🔍 INTERMEDIATE CALCULATIONS:")
print("=" * 70)
print()
print("1️⃣  CORNEAL RADIUS (r):")
print("    r = (k_index - 1) × 1000 / K_avg")
print("    where: k_index = 1.3375 (keratometric index)")
print()
print("2️⃣  CORRECTED AXIAL LENGTH (LCOR):")
print("    If AL ≤ 24.2 mm:  LCOR = AL")
print("    If AL > 24.2 mm:  LCOR = 3.446 + 1.716×AL - 0.0237×AL²")
print()
print("3️⃣  CORNEAL HEIGHT H2 (Sheard's modification):")
print("    H2 = -10.326 + 0.32630×LCOR + 0.13533×K_avg")
print()
print("4️⃣  ESTIMATED ANTERIOR CHAMBER DEPTH (ACDest):")
print("    ACD_const = 0.62467×A_constant - 68.747")
print("    offset = ACD_const - 3.336")
print("    ACDest = H2 + offset")
print()
print("5️⃣  OPTICAL AXIAL LENGTH (Lopt):")
print("    RETHICK = 0.65696 - 0.02029×AL  (retinal thickness)")
print("    Lopt = AL + RETHICK")
print()
print()
print("✓ SRK/T2 formula defined and ready for use")

SRK/T2 FORMULA (Sheard et al. 2010)

📐 MAIN FORMULA:
----------------------------------------------------------------------

         1000·nₐ·(nₐ·r - nc₋₁·Lopt) - P·(Lopt - ACDest)·(nₐ·r - nc₋₁·ACDest)
REF = ───────────────────────────────────────────────────────────────────────────
       nₐ·(V·(nₐ·r - nc₋₁·Lopt) + Lopt·r) - 0.001·P·(Lopt - ACDest)·(V·(nₐ·r - nc₋₁·ACDest) + ACDest·r)


📖 VARIABLE DEFINITIONS:

INPUT VARIABLES:
-----------------------------------
• AL         → Axial length of the eye (mm)
• K_avg      → Average keratometry [(Ks + Kf)/2] (diopters)
• IOL_power  → Implanted intraocular lens power (diopters)
• A_constant → IOL-specific A-constant (dimensionless)

PHYSICAL CONSTANTS:
-----------------------------------
• nₐ = 1.336     → Refractive index of aqueous and vitreous
• nc = 1.333     → Corneal refractive index
• nc₋₁ = 0.333   → nc - 1 (corneal refractive power)
• k_index = 1.3375 → Keratometric index (for K to radius conversion)
• V = 12 mm      → Vertex dista

In [3]:
print("CALCULATING SRK/T2 PREDICTIONS...")
print("-" * 70)

# Calculate predictions for all patients
df['SRKT2_Prediction'] = df.apply(
    lambda row: calculate_SRKT2(
        AL=row['Bio-AL'],
        K_avg=row['K_avg'],
        IOL_power=row['IOL Power'],
        A_constant=row['A-Constant']
    ), axis=1
)

# Calculate prediction errors
df['Prediction_Error'] = df['PostOP Spherical Equivalent'] - df['SRKT2_Prediction']
df['Absolute_Error'] = abs(df['Prediction_Error'])

print(f"✓ Predictions calculated for {len(df)} patients")

# Calculate metrics
mae = df['Absolute_Error'].mean()
me = df['Prediction_Error'].mean()
std = df['Prediction_Error'].std()
median_ae = df['Absolute_Error'].median()

print("\n📊 SRK/T2 FORMULA PERFORMANCE METRICS:")
print("=" * 70)
print(f"  Mean Absolute Error (MAE):     {mae:.4f} D")
print(f"  Mean Error (ME):                {me:+.4f} D")
print(f"  Standard Deviation (SD):        {std:.4f} D")
print(f"  Median Absolute Error:          {median_ae:.4f} D")

# Calculate clinical accuracy
within_025 = (df['Absolute_Error'] <= 0.25).sum() / len(df) * 100
within_050 = (df['Absolute_Error'] <= 0.50).sum() / len(df) * 100
within_075 = (df['Absolute_Error'] <= 0.75).sum() / len(df) * 100
within_100 = (df['Absolute_Error'] <= 1.00).sum() / len(df) * 100

print("\n📈 CLINICAL ACCURACY:")
print("-" * 70)
print(f"  Within ±0.25 D:  {within_025:.1f}% of eyes")
print(f"  Within ±0.50 D:  {within_050:.1f}% of eyes")
print(f"  Within ±0.75 D:  {within_075:.1f}% of eyes")
print(f"  Within ±1.00 D:  {within_100:.1f}% of eyes")

CALCULATING SRK/T2 PREDICTIONS...
----------------------------------------------------------------------
✓ Predictions calculated for 96 patients

📊 SRK/T2 FORMULA PERFORMANCE METRICS:
  Mean Absolute Error (MAE):     1.3591 D
  Mean Error (ME):                -0.2915 D
  Standard Deviation (SD):        1.7471 D
  Median Absolute Error:          1.0311 D

📈 CLINICAL ACCURACY:
----------------------------------------------------------------------
  Within ±0.25 D:  13.5% of eyes
  Within ±0.50 D:  26.0% of eyes
  Within ±0.75 D:  35.4% of eyes
  Within ±1.00 D:  49.0% of eyes


In [4]:
# Correlation analysis between MAE and SRK/T2 parameters
print("\n" + "=" * 70)
print("CORRELATION ANALYSIS: MAE vs SRK/T2 PARAMETERS (SPEARMAN)")
print("=" * 70)

# Calculate intermediate parameters used in the formula for each patient
df['r_corneal'] = (1.3375 - 1) * 1000 / df['K_avg']  # Corneal radius

# LCOR (Corrected Axial Length)
df['LCOR'] = df.apply(lambda row: row['Bio-AL'] if row['Bio-AL'] <= 24.2 
                      else 3.446 + 1.716 * row['Bio-AL'] - 0.0237 * row['Bio-AL']**2, 
                      axis=1)

# H2 (Sheard's Corneal Height)
df['H2'] = -10.326 + 0.32630 * df['LCOR'] + 0.13533 * df['K_avg']

# Estimated ACD
df['ACD_const'] = 0.62467 * df['A-Constant'] - 68.747
df['offset'] = df['ACD_const'] - 3.336
df['ACDest'] = df['H2'] + df['offset']

# Retinal thickness and optical length
df['RETHICK'] = 0.65696 - 0.02029 * df['Bio-AL']
df['Lopt'] = df['Bio-AL'] + df['RETHICK']

# Calculate correlations using Spearman method
correlations = {
    'INPUT PARAMETERS': {
        'Axial Length (AL)': df['Bio-AL'].corr(df['Absolute_Error'], method='spearman'),
        'Average Keratometry (K_avg)': df['K_avg'].corr(df['Absolute_Error'], method='spearman'),
        'IOL Power': df['IOL Power'].corr(df['Absolute_Error'], method='spearman'),
        'A-Constant': df['A-Constant'].corr(df['Absolute_Error'], method='spearman'),
        'CCT': df['CCT'].corr(df['Absolute_Error'], method='spearman')
    },
    'CALCULATED PARAMETERS': {
        'Corneal Radius (r)': df['r_corneal'].corr(df['Absolute_Error'], method='spearman'),
        'Corrected AL (LCOR)': df['LCOR'].corr(df['Absolute_Error'], method='spearman'),
        'Corneal Height H2': df['H2'].corr(df['Absolute_Error'], method='spearman'),
        'Estimated ACD': df['ACDest'].corr(df['Absolute_Error'], method='spearman'),
        'Optical Length (Lopt)': df['Lopt'].corr(df['Absolute_Error'], method='spearman'),
        'Retinal Thickness': df['RETHICK'].corr(df['Absolute_Error'], method='spearman')
    }
}

# Print results
print("\n📊 SPEARMAN CORRELATIONS (ρ) WITH ABSOLUTE ERROR:")
print("-" * 70)

for category, params in correlations.items():
    print(f"\n{category}:")
    print("-" * 35)
    for name, corr in sorted(params.items(), key=lambda x: abs(x[1]), reverse=True):
        sign = "+" if corr > 0 else ""
        strength = ""
        abs_corr = abs(corr)
        if abs_corr >= 0.7:
            strength = " [STRONG]"
        elif abs_corr >= 0.5:
            strength = " [MODERATE]"
        elif abs_corr >= 0.3:
            strength = " [WEAK]"
        else:
            strength = " [VERY WEAK]"
        
        print(f"  {name:30} ρ = {sign}{corr:.4f}{strength}")

# Statistical analysis of significant correlations
print("\n📈 INTERPRETATION:")
print("-" * 70)

# Find strongest correlations
all_corrs = []
for cat, params in correlations.items():
    for name, corr in params.items():
        all_corrs.append((name, corr))

all_corrs.sort(key=lambda x: abs(x[1]), reverse=True)
top_3 = all_corrs[:3]

print("\nTOP 3 STRONGEST CORRELATIONS:")
for i, (name, corr) in enumerate(top_3, 1):
    print(f"{i}. {name}: ρ = {corr:+.4f}")
    if corr > 0:
        print(f"   → Higher {name} values associated with larger errors")
    else:
        print(f"   → Higher {name} values associated with smaller errors")

# Significance testing for main correlations
from scipy import stats

print("\n📊 SIGNIFICANCE TESTING (n = 96):")
print("-" * 70)

# Mapping of names to dataframe columns
param_mapping = {
    'Axial Length (AL)': 'Bio-AL',
    'Average Keratometry (K_avg)': 'K_avg',
    'IOL Power': 'IOL Power',
    'A-Constant': 'A-Constant',
    'CCT': 'CCT',
    'Corneal Radius (r)': 'r_corneal',
    'Corrected AL (LCOR)': 'LCOR',
    'Corneal Height H2': 'H2',
    'Estimated ACD': 'ACDest',
    'Optical Length (Lopt)': 'Lopt',
    'Retinal Thickness': 'RETHICK'
}

for name, corr in top_3:
    # Calculate p-value using scipy.stats.spearmanr
    col_name = param_mapping.get(name)
    if col_name:
        rho, p_value = stats.spearmanr(df[col_name], df['Absolute_Error'])
    else:
        p_value = np.nan
    
    sig = ""
    if p_value < 0.001:
        sig = "***"
    elif p_value < 0.01:
        sig = "**"
    elif p_value < 0.05:
        sig = "*"
    else:
        sig = "ns"
    
    print(f"{name:30} p = {p_value:.4f} {sig}")

print("\nLegend: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")


CORRELATION ANALYSIS: MAE vs SRK/T2 PARAMETERS (SPEARMAN)

📊 SPEARMAN CORRELATIONS (ρ) WITH ABSOLUTE ERROR:
----------------------------------------------------------------------

INPUT PARAMETERS:
-----------------------------------
  Axial Length (AL)              ρ = +0.3429 [WEAK]
  IOL Power                      ρ = -0.2460 [VERY WEAK]
  CCT                            ρ = +0.1887 [VERY WEAK]
  Average Keratometry (K_avg)    ρ = -0.1675 [VERY WEAK]
  A-Constant                     ρ = -0.0307 [VERY WEAK]

CALCULATED PARAMETERS:
-----------------------------------
  Corrected AL (LCOR)            ρ = +0.3429 [WEAK]
  Optical Length (Lopt)          ρ = +0.3429 [WEAK]
  Retinal Thickness              ρ = -0.3429 [WEAK]
  Corneal Height H2              ρ = +0.3134 [WEAK]
  Estimated ACD                  ρ = +0.2997 [VERY WEAK]
  Corneal Radius (r)             ρ = +0.1675 [VERY WEAK]

📈 INTERPRETATION:
----------------------------------------------------------------------

TOP 3 STRONG